In [3]:
import time
import requests
 
import re
from sys import exit
from selenium import webdriver
import zipfile
import platform
import pandas as pd

from zipfile import ZipFile
from stat import S_IXUSR
import os
import unicodedata
import timestring
from urllib.parse import urlparse


class Crawler():
    
    def __init__ (self):
#         self.browser = webdriver.PhantomJS(executable_path=self.get_phantomjs_executable_path())
        pass
        
    def extract_all_with_executable_permission(self,zf, target_dir):
        ZIP_UNIX_SYSTEM = 3

        for info in zf.infolist():
            extracted_path = zf.extract(info, target_dir)

            if info.create_system == ZIP_UNIX_SYSTEM and os.path.isfile(extracted_path):
                unix_attributes = info.external_attr >> 16
                if unix_attributes & S_IXUSR:
                    os.chmod(extracted_path, os.stat(extracted_path).st_mode | S_IXUSR)


    def get_phantomjs_executable_path(self):
        Base_path = "./static/phantomjs/"
        zip_name = "phantomjs-2.1.1-"
        if platform.system() == 'Linux':
            zip_name += "linux"
            if platform.architecture()[0] == '64bit':
                zip_name += "-x64"
                with zipfile.ZipFile(Base_path+zip_name+".zip", 'r') as zip_ref:
                    self.extract_all_with_executable_permission(zip_ref, Base_path)
                return Base_path+zip_name+"/bin/phantomjs"
            else:
                zip_name += "-x86"
                with zipfile.ZipFile(Base_path+zip_name+".zip", 'r') as zip_ref:
                    self.extract_all_with_executable_permission(zip_ref, Base_path)
                return Base_path+zip_name+"/bin/phantomjs"

        elif platform.system() == 'Darwin': # macos
            zip_name += "macosx"
            with zipfile.ZipFile(Base_path+zip_name+".zip", 'r') as zip_ref:
                    self.extract_all_with_executable_permission(zip_ref, Base_path)
            return Base_path+zip_name+"/bin/phantomjs"

        elif platform.system() == 'Windows':
            zip_name += "windows"
            with zipfile.ZipFile(Base_path+zip_name+".zip", 'r') as zip_ref:
                    self.extract_all_with_executable_permission(zip_ref, Base_path)
            return Base_path+zip_name+"/bin/phantomjs.exe"

        else:
            print("os not supported")
            exit()

    def strip_punctuation(self,text):
        if text is None:
            return None
        punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
        return  ''.join(x for x in text if unicodedata.category(x)
                       not in punctutation_cats)

    def scrape_data(self ,url  , path ,depth,flag=0):
        
        base_url = urlparse(url).scheme+"://"+urlparse(url).netloc+"/"
        
        

#         self.browser.get(url)
#         html = self.browser.page_source
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            response = None
            for i in range (3):
                try:
                    response = requests.get(url,headers=headers)
                    break
                except ConnectionError as e:
                    print(e)
                    if i == 2:
                        return e
            html = response.content
            

            if response.status_code == 200:
                file_name = urlparse(url).netloc+"/"+urlparse(url).path.replace("../","").replace("/","")
                file_name = file_name.replace("/","")

                if path[-1] != "/":
                    path+="/"

                if url[-5] != ".html":
                    file_name += ".html"

                

                with open(path+file_name , 'wb') as f:
                    f.write(html)
                
                

                print("flag : ",flag)
                print("url : ",url)
                print("file name : ",path+file_name)
                print('-'*30)

                if flag < depth:
                    soup = BeautifulSoup(html,'lxml')
                    all_as = soup.findAll("a")
                    if all_as:
                        for a in all_as:
                            try:
                                href = a['href']
        #                         print(href)
                                if href:
                                    if href.find("://") == -1:
        #                                 print(base_url+href)
                                        time.sleep(2)
                                        self.scrape_data(base_url+href , path ,depth ,flag+1)
                                    elif urlparse(href).netloc == urlparse(url).netloc:
        #                                 print(href)
                                        time.sleep(2)
                                        self.scrape_data(href , path ,depth ,flag+1)
                            except KeyError as e:
                                print(e)
                                pass
                    else:
                        print("no links")
        #                 print(all_as)
            else:
                print("status_code : " , response.status_code)
        except Exception as e:
            print("error")

In [4]:
base_url = "http://xb.81.cn"

c = Crawler()
c.scrape_data(base_url,"./data",4)

flag :  0
url :  http://xb.81.cn
file name :  ./data/xb.81.cn.html
------------------------------
flag :  1
url :  http://xb.81.cn/#lxwm
file name :  ./data/xb.81.cn.html
------------------------------
flag :  2
url :  http://xb.81.cn/#lxwm
file name :  ./data/xb.81.cn.html
------------------------------
flag :  3
url :  http://xb.81.cn/#lxwm
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#lxwm
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  400
status_code :  400
status_code :  400
status_code :  400
status_code :  400
status_code :  400
status_code :  400
status_code :  400
flag :  4
url :  http://xb.81.cn/#wnfw
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  400
status_code :  400
status_code :  400
flag :  4
url :  http://xb.81.cn/#jlqy
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  400
status_code :  400
status_code :  400
flag :  4
ur

flag :  4
url :  http://xb.81.cn/content/2016-11/18/content_7365577.htm
file name :  ./data/xb.81.cncontent2016-1118content_7365577.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-10/12/content_7296747.htm
file name :  ./data/xb.81.cncontent2016-1012content_7296747.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-08/31/content_7234561.htm
file name :  ./data/xb.81.cncontent2016-0831content_7234561.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2017-12/04/content_7855906.htm
file name :  ./data/xb.81.cncontent2017-1204content_7855906.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-03/11/content_6955799.htm
file name :  ./data/xb.81.cncontent2016-0311content_6955799.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-02/22/content_6923019.htm
file name :  ./data/xb.81.cncontent2016-0222content_6923019.htm.html

flag :  4
url :  http://xb.81.cn/content/2017-06/15/content_7640426.htm
file name :  ./data/xb.81.cncontent2017-0615content_7640426.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2017-06/07/content_7631190.htm
file name :  ./data/xb.81.cncontent2017-0607content_7631190.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2017-06/07/content_7631180.htm
file name :  ./data/xb.81.cncontent2017-0607content_7631180.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2017-04/10/content_7557244.htm
file name :  ./data/xb.81.cncontent2017-0410content_7557244.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-11/25/content_7377048.htm
file name :  ./data/xb.81.cncontent2016-1125content_7377048.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-11/24/content_7375086.htm
file name :  ./data/xb.81.cncontent2016-1124content_7375086.htm.html

status_code :  400
status_code :  404
status_code :  400
flag :  4
url :  http://xb.81.cn/content/2016-09/28/content_7280396.htm
file name :  ./data/xb.81.cncontent2016-0928content_7280396.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-04/27/content_7025498.htm
file name :  ./data/xb.81.cncontent2016-0427content_7025498.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-03/23/content_6972757.htm
file name :  ./data/xb.81.cncontent2016-0323content_6972757.htm.html
------------------------------
status_code :  400
'href'
status_code :  400
status_code :  400
'href'
status_code :  400
flag :  4
url :  http://xb.81.cn/content/2017-11/13/content_7821859.htm
file name :  ./data/xb.81.cncontent2017-1113content_7821859.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2017-09/21/content_7764884.htm
file name :  ./data/xb.81.cncontent2017-0921content_7764884.htm.html
------------------------

flag :  4
url :  http://xb.81.cn/content/2016-05/20/content_7063534.htm
file name :  ./data/xb.81.cncontent2016-0520content_7063534.htm.html
------------------------------
status_code :  400
flag :  4
url :  http://xb.81.cn/content/2017-11/23/content_7839065.htm
file name :  ./data/xb.81.cncontent2017-1123content_7839065.htm.html
------------------------------
status_code :  400
status_code :  400
'href'
status_code :  400
flag :  4
url :  http://xb.81.cn/content/2016-09/29/content_7282595.htm
file name :  ./data/xb.81.cncontent2016-0929content_7282595.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-05/19/content_7062019.htm
file name :  ./data/xb.81.cncontent2016-0519content_7062019.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-05/05/content_7036387.htm
file name :  ./data/xb.81.cncontent2016-0505content_7036387.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/content/2016-05/05/conte

...

flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  404
status_code :  404
flag :  3
url :  http://xb.81.cn/content/2017-06/07/content_7631190.htm
file name :  ./data/xb.81.cncontent2017-0607content_7631190.htm.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  400
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404


status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  400
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------

flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  400
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  404
st

flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
status_code :  404
status_code :  404
status_code :  404
status_code :  404
status_code :  403
status_code :  404
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
------------------------------
flag :  4
url :  http://xb.81.cn/#
file name :  ./data/xb.81.cn.html
-----------------

KeyboardInterrupt: 

In [ ]:
files = []

# domain_id = Domain(url=base_url)

for (dirpath, dirnames, filenames) in os.walk(".data"):#../patents data
    count = 0
    for file in filenames:
            files.append(file)
            html = ""
            with open("./data/"+file , "rb") as f:
                html = f.read()
            soup = BeautifulSoup(html,"lxml")
            all_ps = soup.findAll("p")
            if all_ps:
                for p in all_ps:
                    text = p.getText(strip=True)
                    # regix replace words with tag
                    # for each word found save keywords_pargraphs
                    for word in keywords:
                        if
                    html = re.sub('foo','bar', line.rstrip())
                    # save pargraph
                
    break